## Recalling the last time I studied this

Let me explain if you didn't read the project proposal. Several years before taking this AI/ML course, I performed a study to explain why electricity demand varies by day. It's a very basic question that if one can't answer then one doesn't have any business explaining anything else that goes on in electricity markets. I inherited the study using Heating Degree Days and Cooling Degree Days, and reimplemented it with a few more predictors like day of week and holidays.

I used Excel's multiple linear regression on just a few summarized raw features, and it worked pretty well. I thought it was hot stuff, but my boss at the time, with a Ph.D. in Econometrics, said "don't show this to anybody before seeing me". Unfortunate events intevened and I was unable to ever have her review it, so I still don't know what errors she saw, but here I'll try to recreate it here **making the same mistakes**. 

In [1]:
import pandas as pd
import numpy as np
target_df = pd.read_pickle("dataframes/target_df.pickle.gz", compression="infer")

In [2]:
target_df = target_df[['sum_spp_load', 'is_Thursday', 'is_Saturday', 'is_Sunday', 'is_holiday', 'SUM_CDD', 'SUM_HDD' ]]
target_df


,sum_spp_load,is_Thursday,is_Saturday,is_Sunday,is_holiday,SUM_CDD,SUM_HDD
0,502184.942993,1,0,0,0,20.10,8.76
1,612695.032412,0,0,0,0,38.88,0.00
2,566229.834662,0,1,0,0,61.38,0.00
3,552166.723416,0,0,1,0,55.62,0.00
4,596751.461994,0,0,0,0,52.56,0.00
...,...,...,...,...,...,...,...
1549,648146.557000,0,1,0,0,132.12,0.00
1550,661993.471000,0,0,1,0,169.74,0.00
1551,752393.427000,0,0,0,0,197.10,0.00
1552,746139.564000,0,0,0,0,199.08,0.00


In [3]:
## separate independent and dependent variables


In [4]:
# Referencing https://www.analyticsvidhya.com/blog/2021/05/multiple-linear-regression-using-python-and-scikit-learn/

# separate the target, or dependent, attribute from the predicting attributes
X = target_df.drop('sum_spp_load',axis=1)
# separate the target attribute into Y for model training 
y = target_df['sum_spp_load']

In [5]:
# from https://towardsdatascience.com/simple-and-multiple-linear-regression-in-python-c928425168f9


In [6]:
import statsmodels.api as sm # import statsmodels 

#X = df["RM"] ## X usually means our input variables (or independent variables)
#y = target["MEDV"] ## Y usually means our output/dependent variable
X = sm.add_constant(X) ## let's add an intercept (beta_0) to our model

# Note the difference in argument order
model = sm.OLS(y, X).fit() ## sm.OLS(output, input)
predictions = model.predict(X)

# Print out the statistics
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           sum_spp_load   R-squared:                       0.892
Model:                            OLS   Adj. R-squared:                  0.891
Method:                 Least Squares   F-statistic:                     2118.
Date:                Fri, 12 Nov 2021   Prob (F-statistic):               0.00
Time:                        20:20:54   Log-Likelihood:                -18384.
No. Observations:                1553   AIC:                         3.678e+04
Df Residuals:                    1546   BIC:                         3.682e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const        5.771e+05   2071.688    278.587      0.000    5.73e+05    5.81e+05
is_Thursday  5026.1187   2517.695      1.996      0.046      87.661    9964.577
is_Saturday -3.273e+04   2526.305    -12.954      0.000   -3.77e+04   -2.78e+04
is_Sunday   -4.708e+04   2519.763    -18.686      0.000    -5.2e+04   -4.21e+04
is_holiday  -5.588e+04   5537.587    -10.092      0.000   -6.67e+04    -4.5e+04
SUM_CDD       792.9042     13.345     59.417      0.000     766.728     819.080
SUM_HDD      3579.4655     32.909    108.769      0.000    3514.915    3644.016
==============================================================================
Omnibus:                     1064.317   Durbin-Watson:                   0.766
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            98724.899
Skew:                          -2.386   Prob(JB):                         0.00
Kurtosis:                      41.767   Cond. No.                         769.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## I got better results by adding a sequence number as a predictor, because historically electric demand has grown over time.
The sequence number can cause a slightly better fit on growth over time. However, there is nothing that suggests that growth is linear.  

In [7]:
X = target_df.drop('sum_spp_load',axis=1)
X = sm.add_constant(X) ## let's add an intercept (beta_0) to our model
X['sequence_num'] = X.index

# Note the difference in argument order
model = sm.OLS(y, X).fit() ## sm.OLS(output, input)
predictions = model.predict(X)

# Print out the statistics
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           sum_spp_load   R-squared:                       0.905
Model:                            OLS   Adj. R-squared:                  0.904
Method:                 Least Squares   F-statistic:                     2091.
Date:                Fri, 12 Nov 2021   Prob (F-statistic):               0.00
Time:                        20:20:54   Log-Likelihood:                -18285.
No. Observations:                1553   AIC:                         3.659e+04
Df Residuals:                    1545   BIC:                         3.663e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           5.6e+05   2276.137    246.033      0.000    5.56e+05    5.64e+05
is_Thursday   5121.6985   2363.118      2.167      0.030     486.440    9756.957
is_Saturday  -3.269e+04   2371.191    -13.787      0.000   -3.73e+04    -2.8e+04
is_Sunday    -4.714e+04   2365.053    -19.932      0.000   -5.18e+04   -4.25e+04
is_holiday   -5.542e+04   5197.678    -10.663      0.000   -6.56e+04   -4.52e+04
SUM_CDD        773.6150     12.596     61.418      0.000     748.908     798.322
SUM_HDD       3516.5932     31.192    112.742      0.000    3455.411    3577.775
sequence_num    26.0567      1.799     14.487      0.000      22.529      29.585
==============================================================================
Omnibus:                     1326.955   Durbin-Watson:                   0.845
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           224669.903
Skew:                          -3.245   Prob(JB):                         0.00
Kurtosis:                      61.566   Cond. No.                     5.86e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.86e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## TODO: elaborate on the warnings above. To the best of my memory, these numbers fairly well match my previous results. 

## TODO:  plot modeled vs. actual. 

## What if I used EVERYTHING in my target DF all at once?
I know, curse of dimensionality, strong colinearity, etc.  Not advised. But what does the model do? 

In [8]:
# what if I used EVERYTHING in my original DF? 
target_df = pd.read_pickle("dataframes/target_df.pickle.gz", compression="infer").dropna()
y = target_df['sum_spp_load']
X = target_df.drop(['sum_spp_load', 'opday'],axis=1)
X = sm.add_constant(X) ## let's add an intercept (beta_0) to our model
X['sequence_num'] = X.index

model = sm.OLS(y, X).fit() ## sm.OLS(output, input)
predictions = model.predict(X)

# Print out the statistics
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           sum_spp_load   R-squared:                       0.923
Model:                            OLS   Adj. R-squared:                  0.921
Method:                 Least Squares   F-statistic:                     443.0
Date:                Fri, 12 Nov 2021   Prob (F-statistic):               0.00
Time:                        20:20:54   Log-Likelihood:                -18116.
No. Observations:                1553   AIC:                         3.632e+04
Df Residuals:                    1511   BIC:                         3.654e+04
Df Model:                          41                                         
Covariance Type:            nonrobust                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
const                                 288.7464      8.331     34.657      0.000     272.404     305.089
is_Friday                            1.102e+04   1807.697      6.094      0.000    7469.881    1.46e+04
is_Monday                            3881.3817   1821.680      2.131      0.033     308.092    7454.672
is_Saturday                         -2.204e+04   1791.690    -12.299      0.000   -2.56e+04   -1.85e+04
is_Sunday                           -3.664e+04   1787.887    -20.492      0.000   -4.01e+04   -3.31e+04
is_Thursday                          1.636e+04   1810.929      9.032      0.000    1.28e+04    1.99e+04
is_Tuesday                            1.51e+04   1787.532      8.448      0.000    1.16e+04    1.86e+04
is_Wednesday                         1.261e+04   1793.523      7.029      0.000    9087.834    1.61e+04
holiday_Christmas Day               -8252.7669   1.27e+04     -0.647      0.517   -3.33e+04    1.68e+04
holiday_Christmas Day (Observed)     1119.1919   2.66e+04      0.042      0.966   -5.11e+04    5.33e+04
holiday_Christmas Eve                1.784e+04   1.27e+04      1.400      0.162   -7164.912    4.28e+04
holiday_Day After Thanksgiving      -1.781e+04   1.28e+04     -1.387      0.166    -4.3e+04    7381.255
holiday_Independence Day            -7836.9495   1.41e+04     -0.555      0.579   -3.56e+04    1.99e+04
holiday_Independence Day (Observed)  1.632e-11   5.44e-11      0.300      0.764   -9.03e-11    1.23e-10
holiday_Labor Day                   -1.083e+04   1.41e+04     -0.768      0.443   -3.85e+04    1.68e+04
holiday_Memorial Day                -1.551e+04    1.4e+04     -1.104      0.270   -4.31e+04     1.2e+04
holiday_New Year's Day               5147.5656   1.41e+04      0.366      0.714   -2.24e+04    3.27e+04
holiday_New Year's Day (Observed)      44.3460   2.66e+04      0.002      0.999   -5.21e+04    5.22e+04
holiday_Thanksgiving                -8270.8855   1.29e+04     -0.643      0.520   -3.35e+04    1.69e+04
is_holiday                          -4.435e+04   5214.181     -8.506      0.000   -5.46e+04   -3.41e+04
TMIN_USW00013967                      130.8300    585.963      0.223      0.823   -1018.556    1280.216
TMIN_USW00013968                      728.7863    549.897      1.325      0.185    -349.857    1807.429
TMIN_USW00013996                     -838.4932    509.189     -1.647      0.100   -1837.284     160.298
TMIN_USW00014939                      344.9725    441.593      0.781      0.435    -521.228    1211.173
TMIN_USW00023047                     1037.9562    433.529      2.394      0.017     187.573    1888.339
TMIN_USW00024011                     -798.7911    326.942     -2.443      0.015   -1440.099    -157.483
TMAX_USW00013967                      -55.7285    557.794     -0.10

## Todo:  save predictions for all points; compare later to ML models


## What if I normalize the predictors and target first? 


In [9]:
# what if I used EVERYTHING in my original DF? 
target_df = pd.read_pickle("dataframes/target_df.pickle.gz", compression="infer").dropna()
y = target_df['sum_spp_load']
X = target_df.drop(['sum_spp_load', 'opday'],axis=1)
X['sequence_num'] = X.index

# normalize predictor variables
X=(X-X.min())/(X.max()-X.min() + 0.0001)  # small constant to prevent div/0

X = sm.add_constant(X) ## let's add an intercept (beta_0) to our model

# normalize target variable
y=(y-y.min())/(y.max()-y.min())

model = sm.OLS(y, X).fit() ## sm.OLS(output, input)
predictions = model.predict(X)

# Print out the statistics
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           sum_spp_load   R-squared:                       0.923
Model:                            OLS   Adj. R-squared:                  0.921
Method:                 Least Squares   F-statistic:                     443.0
Date:                Fri, 12 Nov 2021   Prob (F-statistic):               0.00
Time:                        20:20:54   Log-Likelihood:                 3058.8
No. Observations:                1553   AIC:                            -6034.
Df Residuals:                    1511   BIC:                            -5809.
Df Model:                          41                                         
Covariance Type:            nonrobust                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
const                                   0.1529      0.008     18.098      0.000       0.136       0.169
is_Friday                               0.0350      0.002     14.164      0.000       0.030       0.040
is_Monday                               0.0264      0.002     10.575      0.000       0.022       0.031
is_Saturday                            -0.0046      0.003     -1.849      0.065      -0.010       0.000
is_Sunday                              -0.0221      0.002     -9.038      0.000      -0.027      -0.017
is_Thursday                             0.0414      0.002     16.815      0.000       0.037       0.046
is_Tuesday                              0.0399      0.002     16.192      0.000       0.035       0.045
is_Wednesday                            0.0369      0.002     15.003      0.000       0.032       0.042
holiday_Christmas Day                  -0.0099      0.015     -0.647      0.517      -0.040       0.020
holiday_Christmas Day (Observed)        0.0013      0.032      0.042      0.966      -0.061       0.064
holiday_Christmas Eve                   0.0214      0.015      1.400      0.162      -0.009       0.051
holiday_Day After Thanksgiving         -0.0213      0.015     -1.387      0.166      -0.052       0.009
holiday_Independence Day               -0.0094      0.017     -0.555      0.579      -0.043       0.024
holiday_Independence Day (Observed)  -3.64e-19   1.16e-17     -0.031      0.975   -2.31e-17    2.23e-17
holiday_Labor Day                      -0.0130      0.017     -0.768      0.443      -0.046       0.020
holiday_Memorial Day                   -0.0186      0.017     -1.104      0.270      -0.052       0.014
holiday_New Year's Day                  0.0062      0.017      0.366      0.714      -0.027       0.039
holiday_New Year's Day (Observed)    5.315e-05      0.032      0.002      0.999      -0.062       0.063
holiday_Thanksgiving                   -0.0099      0.015     -0.643      0.520      -0.040       0.020
is_holiday                             -0.0532      0.006     -8.506      0.000      -0.065      -0.041
TMIN_USW00013967                        0.0071      0.032      0.223      0.823      -0.055       0.070
TMIN_USW00013968                        0.0387      0.029      1.325      0.185      -0.019       0.096
TMIN_USW00013996                       -0.0501      0.030     -1.647      0.100      -0.110       0.010
TMIN_USW00014939                        0.0227      0.029      0.781      0.435      -0.034       0.080
TMIN_USW00023047                        0.0544      0.023      2.394      0.017       0.010       0.099
TMIN_USW00024011                       -0.0557      0.023     -2.443      0.015      -0.100      -0.011
TMAX_USW00013967                       -0.0034      0.034     -0.10

## Exactly the same results! 

OLS must be robust to normalization problems.  I can however now see the relative impact of each predictor variable by its coefficient magnitude.

In [11]:
## Next steps:  break out scikit-learn, and see what we can do with dimensionality reduction and modern altorithms.